# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [66]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.info()
df_market_data.head(10)
# DataFrame below shows percentage price changes over various time frames for 41 different cryptocurrencies
# There are 41 cryptocurrencies and 7 features, or factors.  The features are percentage price change for 41 cryptocurrencies over 7 different time frames.

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, bitcoin to digibyte
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price_change_percentage_24h   41 non-null     float64
 1   price_change_percentage_7d    41 non-null     float64
 2   price_change_percentage_14d   41 non-null     float64
 3   price_change_percentage_30d   41 non-null     float64
 4   price_change_percentage_60d   41 non-null     float64
 5   price_change_percentage_200d  41 non-null     float64
 6   price_change_percentage_1y    41 non-null     float64
dtypes: float64(7)
memory usage: 2.6+ KB


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()
# DataFrame below shows aggregate summary statistics over various time frames for a basket of 41 cryptocurrencies

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)
# Plot below shows percentage price changes over various time frames for 41 different cryptocurrencies

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [28]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# StandardScaler() below standardizes each data series around its mean as a unit normal (standard unit normal: mean==0, standard deviation==1)
scaled_data = StandardScaler().fit_transform(df_market_data)
# Examine first five rows of scaled data array
display(scaled_data[:5])

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [6]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coin_id column as the index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()
# DataFrame below shows percentage price changes over various time frames, where each time frame series is scaled as a standard unit normal, for 41 different cryptocurrencies

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [7]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k_values = list(range(1,11)) # dir(k_values) returns attributes and methods for list object
print(k_values)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [8]:
# Create an empty list to store the inertia values
inertia_values = [] # or 'inertia_values = list()'
print(inertia_values)

[]


In [9]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
# Initialize random state as rand_int prior to entering loop
rand_int=45
for each_k in k_values:
    km_model_crypto_scaled = KMeans(each_k, random_state = rand_int) # Instantiate the KMeans algorithm object for n_clusters = each_k and specified random_state
    km_model_crypto_scaled.fit(df_market_data_scaled) # Fit the KMeans algorithm for various cluster numbers 'each_k' to the standard unit normal scaled crypto dataset to create the model (algo+data = model)
    inertia_values.append(km_model_crypto_scaled.inertia_) # Append the KMeans model's inertia for each cluster count evaluated
print(inertia_values)

[287.0, 195.82021818036043, 123.19048183836958, 79.02243535120975, 63.85866780584266, 52.933558921015006, 44.55027802077481, 37.13860998272166, 32.4370030616485, 28.64258346738439]


In [42]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_data = {'k': k_values, 'inertia': inertia_values}
#print(elbow_curve_data)
# Create a DataFrame with the data to plot the Elbow curve
elbow_curve_data_df = pd.DataFrame(elbow_curve_data).set_index(['k'])
print(elbow_curve_data_df)

       inertia
k             
1   287.000000
2   195.820218
3   123.190482
4    79.022435
5    63.858668
6    52.933559
7    44.550278
8    37.138610
9    32.437003
10   28.642583


In [43]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# elbow_curve_data_df.plot(title='Elbow Method for Cryptocurrency Dataset to Determine Optimal Number of Clusters', xticks=k_values) # Standard plot
elbow_curve_data_df.hvplot.line(title='Elbow Method for Cryptocurrency Dataset to Determine Optimal Number of Clusters',
                                xlabel='number of clusters, k', ylabel='inertia value', width=700, height=400, xticks=k_values)

:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** `While there are a number of different methods to determine the optimal number of clusters, or k-value, for KMeans models, using the Elbow Method in this case,
the optimal k-value for our KMeans model is 4.`

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [12]:
# Initialize the K-Means model using the best value for k
k_value_optimal = 4
rand_int = 47
km_model_crypto_scaled_optimal = KMeans(n_clusters = k_value_optimal, random_state = rand_int) # Instantiate the KMeans algorithm object for model's optimal k-value and specified random_state

In [13]:
# Fit the K-Means model using the scaled data
km_model_crypto_scaled_optimal.fit(df_market_data_scaled) # Fit the KMeans algorithm to the standard unit normal scaled crypto dataset with the model's optimal k-value to create the best-fit KMeans model (algo+data = model)

# Examine the inertia of the optimized model
print(km_model_crypto_scaled_optimal.inertia_)

79.02243535120975


In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled data
km_model_crypto_scaled_optimal_predictions = km_model_crypto_scaled_optimal.predict(df_market_data_scaled)

# View the resulting array of cluster predictions.
display(km_model_crypto_scaled_optimal_predictions)

array([0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 2, 0, 3, 3, 1, 3, 3, 3, 3],
      dtype=int32)

In [15]:
# Create a copy of the DataFrame
df_market_data_with_predictions = df_market_data.copy() #!Important: Use copy() notation, not merely assignment, otherwise changes to the assigned df will also reflect in the original df
display(df_market_data_with_predictions.head(8))

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756


In [16]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_with_predictions['km_cluster'] = km_model_crypto_scaled_optimal_predictions

# Display sample data
display(df_market_data_with_predictions.head(8))

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,km_cluster
coin_id,,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,0
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,0
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,3
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,3
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,0
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195,0
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186,0
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756,0


In [17]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_data_with_predictions.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_7d', by='km_cluster',
                                               hover_cols = 'coin_id', width=700, height=400,
                                               xlabel='24-hr Percentage Price Change', ylabel='7-day Percentage Price Change',
                                               title='Cryptocurrency Scatter Plot of 7-Day and 24-Hour Percentage Price Change by KMeans Cluster',
                                               fontscale=0.85)

:NdOverlay   [km_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [18]:
# Create a PCA model instance and set `n_components=3`.
# Principal Component Analysis (PCA) model proposed to reduce cryptocurrency dataset features from 7 to 3
pca_model_crypto_scaled = PCA(n_components=3) # Instantiate the PCA model object for 3 principal components

In [25]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
# Use conditioned, or scaled, cryptocurrency market data in Principal Component Analysis (PCA)
pca_model_crypto_scaled_fit = pca_model_crypto_scaled.fit_transform(df_market_data_scaled) # As with KMeans model fit, PCA model fit returns an array
#display(pca_model_fit_crypto_scaled)

# View the first five rows of the Array using slice notation.
display(pca_model_crypto_scaled_fit[:5])

# Alternatively, View the first five rows of the DataFrame.
# Convert PCA model fit output from array to DataFrame
pca_model_crypto_scaled_fit_df = pd.DataFrame(pca_model_crypto_scaled_fit, columns=['PC_1', 'PC_2', 'PC_3'])
#display(pca_model_crypto_scaled_fit_df.info())
display(pca_model_crypto_scaled_fit_df.head(5))

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

,PC_1,PC_2,PC_3
0,-0.600667,0.842760,0.461595
1,-0.458261,0.458466,0.952877
2,-0.433070,-0.168126,-0.641752
3,-0.471835,-0.222660,-0.479053
4,-1.157800,2.041209,1.859715


In [20]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_model_crypto_scaled_explained_var = pca_model_crypto_scaled.explained_variance_
pca_model_crypto_scaled_explained_var_ratio = pca_model_crypto_scaled.explained_variance_ratio_
display(f"Explained Variance by principal component is: {pca_model_crypto_scaled_explained_var}")
display(f"Explained Variance Ratio by principal component is: {pca_model_crypto_scaled_explained_var_ratio}")
display(f"Total Explained Variance by the three components is: {sum(pca_model_crypto_scaled_explained_var_ratio)*100:.1f}%")

'Explained Variance by principal component is: [2.6689967  2.48978331 1.26307212]'

'Explained Variance Ratio by principal component is: [0.3719856  0.34700813 0.17603793]'

'Total Explained Variance by the three components is: 89.5%'

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total variance explained by the three principal components is `89.5%`

In [40]:
# Create a new DataFrame with the PCA data.
pca_model_crypto_df = pd.DataFrame(pca_model_crypto_scaled_fit, columns=['PC_1', 'PC_2', 'PC_3'])
#display(pca_model_crypto_df.head())

# Copy the crypto names from the original data
pca_model_crypto_df['coin_id'] = df_market_data.index

# Set the coin_id column as index
pca_model_crypto_df = pca_model_crypto_df.set_index(['coin_id'])

# Display sample data
# PCA dataset was derived from scaled data series and therefore does not itself need to be rescaled
display(pca_model_crypto_df.head())

,PC_1,PC_2,PC_3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [37]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k_values_pca = list(range(1,11))
print(k_values_pca)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [38]:
# Create an empty list to store the inertia values
inertia_values_pca = [] # or 'inertia_values = list()'
print(inertia_values_pca)

[]


In [39]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `pca_model_crypto_df`
# 3. Append the model.inertia_ to the inertia list
# Initialize random state as rand_int prior to entering loop
rand_int=17
for each_k in k_values_pca:
    km_model_crypto_scaled_pca = KMeans(each_k, random_state = rand_int) # Instantiate the KMeans algorithm object for n_clusters = each_k and specified random_state
    km_model_crypto_scaled_pca.fit(pca_model_crypto_df) # Fit the KMeans algorithm for various cluster numbers 'each_k' to the crypto PCA dataset to create the KMeans model (algo+data = model)
    inertia_values_pca.append(km_model_crypto_scaled_pca.inertia_) # Append the KMeans model's inertia for each cluster count evaluated
print(inertia_values_pca)

[256.8740855678926, 165.90199402036012, 93.77462568057298, 49.66549665179738, 37.87874703346251, 27.618971787957445, 21.16397791177445, 17.091636643864753, 13.681139692992758, 10.572379799092946]


In [44]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_data_pca = {'k': k_values_pca, 'inertia': inertia_values_pca}
#print(elbow_curve_data_pca)
# Create a DataFrame with the data to plot the Elbow curve
elbow_curve_data_pca_df = pd.DataFrame(elbow_curve_data_pca).set_index(['k'])
print(elbow_curve_data_pca_df)

       inertia
k             
1   256.874086
2   165.901994
3    93.774626
4    49.665497
5    37.878747
6    27.618972
7    21.163978
8    17.091637
9    13.681140
10   10.572380


In [48]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve_data_pca_df.hvplot.line(title='Elbow Method for Cryptocurrency *PCA* Dataset to Determine Optimal Number of Clusters',
                                xlabel='number of clusters, k', ylabel='inertia value', width=700, height=400, xticks=k_values_pca,
                                fontscale=0.9)

:Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** `The best value for the KMeans cluster count parameter for the PCA data, using the Elbow Method, is 4.`


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** `The best value for the KMeans cluster count parameter, using the Elbow Method, is coincidentally also 4, the same as used with the original scaled data prior to PCA feature reduction.`

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [52]:
# Initialize the K-Means model using the best value for k
k_value_pca_optimal = 4
rand_int = 56
km_model_crypto_scaled_pca_optimal = KMeans(n_clusters = k_value_pca_optimal, random_state = rand_int) # Instantiate the KMeans algorithm object for model's optimal k-value and specified random_state

KMeans(n_clusters=4, random_state=56)

In [53]:
# Fit the K-Means model using the PCA data
km_model_crypto_scaled_pca_optimal.fit(pca_model_crypto_df) # Fit the KMeans algorithm to the crypto PCA dataset with the model's optimal k-value to create the best-fit KMeans model (algo+data = model)

# Examine the inertia of the optimized model
print(km_model_crypto_scaled_pca_optimal.inertia_)

49.66549665179738


In [54]:
# Predict the clusters to group the cryptocurrencies using the PCA data
km_model_crypto_scaled_pca_optimal_predictions = km_model_crypto_scaled_pca_optimal.predict(pca_model_crypto_df)

# View the resulting array of cluster predictions.
display(km_model_crypto_scaled_pca_optimal_predictions)

array([0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 0, 2, 2, 3, 2, 2, 2, 2],
      dtype=int32)

In [56]:
# Create a copy of the DataFrame with the PCA data
pca_model_crypto_with_predictions_df = pca_model_crypto_df.copy()
display(pca_model_crypto_with_predictions_df.head(8))

# Add a new column to the DataFrame with the predicted clusters
pca_model_crypto_with_predictions_df['km_cluster_pca'] = km_model_crypto_scaled_pca_optimal_predictions

# Display sample data
display(pca_model_crypto_with_predictions_df.head(8))

,PC_1,PC_2,PC_3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715
binancecoin,-0.516534,1.388377,0.804071
chainlink,-0.450711,0.517699,2.846143
cardano,-0.345600,0.729439,1.478013


,PC_1,PC_2,PC_3,km_cluster_pca
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,0
ethereum,-0.458261,0.458466,0.952877,0
tether,-0.433070,-0.168126,-0.641752,2
ripple,-0.471835,-0.222660,-0.479053,2
bitcoin-cash,-1.157800,2.041209,1.859715,0
binancecoin,-0.516534,1.388377,0.804071,0
chainlink,-0.450711,0.517699,2.846143,0
cardano,-0.345600,0.729439,1.478013,0


In [59]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_model_crypto_with_predictions_df.hvplot.scatter(x='PC_1', y='PC_2', by='km_cluster_pca',
                                               hover_cols = 'coin_id', width=700, height=400,
                                               xlabel='Principal Component 1', ylabel='Principal Component 2',
                                               title='Cryptocurrency Principal Component Analysis Scatter Plot of PC1 and PC2 by KMeans Cluster',
                                               fontscale=0.85)

:NdOverlay   [km_cluster_pca]
   :Scatter   [PC_1]   (PC_2,coin_id)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [71]:
# Composite plot to contrast the Elbow curves
(elbow_curve_data_df.hvplot.line(title='Elbow Method for Cryptocurrency Original Scaled Dataset to Determine Optimal Number of Clusters',
                                xlabel='number of clusters, k', ylabel='inertia value', width=700, height=400, xticks=k_values, fontscale=0.80)) + \
                                (elbow_curve_data_pca_df.hvplot.line(title='Elbow Method for Cryptocurrency *PCA* Dataset to Determine Optimal Number of Clusters',
                                xlabel='number of clusters, k', ylabel='inertia value', width=700, height=400, xticks=k_values_pca, fontscale=0.80))

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [68]:
# Composite plot to contrast the clusters
(df_market_data_with_predictions.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_7d', by='km_cluster',
                                               hover_cols = 'coin_id', width=700, height=400,
                                               xlabel='24-hr Percentage Price Change', ylabel='7-day Percentage Price Change',
                                               title='Cryptocurrency Scatter Plot of 7-Day and 24-Hour Percentage Price Change by KMeans Cluster',
                                               fontscale=0.85)) + \
(pca_model_crypto_with_predictions_df.hvplot.scatter(x='PC_1', y='PC_2', by='km_cluster_pca', 
                                                    hover_cols = 'coin_id', width=700, height=400, 
                                                    xlabel='Principal Component 1', ylabel='Principal Component 2', 
                                                    title='Cryptocurrency Principal Component Analysis Scatter Plot of PC1 and PC2 by KMeans Cluster', 
                                                    fontscale=0.85))

:Layout
   .NdOverlay.I  :NdOverlay   [km_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [km_cluster_pca]
      :Scatter   [PC_1]   (PC_2,coin_id)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** # YOUR ANSWER HERE!